In [110]:
import pandas as pd
import re
# all sentences
# df = pd.read_csv("all_sents_aug.csv").dropna().reset_index()

# only canonical sentences in eng
df = pd.read_csv("canonical_sents_aug.csv").dropna().reset_index()

In [111]:
sentences = df["preprocessed_text"]
sentences = [re.sub(r'\s([?.!"](?:\s|$))', r'\1', x).strip() for x in sentences]
labels = [1] * len(sentences) # synthetic broca data

In [112]:
len(sentences)

22193

In [113]:
sentences

['So i have two children.',
 'One of my children is adopt.',
 'So i tell story my son being bear.',
 'So i guess we talk my daughter and adoption.',
 'We try have children many years and not get.',
 'But i was bind and determine be mom.',
 'Then we start pursue adoption route.',
 'So you know we say we just wait her.',
 'Also i adopt child.',
 'But you know lot people say.',
 'It not feel my child.',
 'That she like adopt child foster system.',
 'Boy play soccer his yard.',
 'One time he kick it bit too.',
 'He kick it his front window.',
 'His dad sit living room.',
 'Ball come window and knock lamp.',
 'So boy get go school.',
 'His mother say.',
 'Johnny you need take umbrella.',
 'It go rain today.',
 'He say.',
 'I not need umbrella.',
 'He leave.',
 'As he get school it start rain.',
 'Then it start rain.',
 'He turn around.',
 'He go back home and tell his mom.',
 'It start rain.',
 'He end up take umbrella and walk back to school.',
 'Girl ride along her tricycle her yard.',
 '

# BERT

In [114]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch     

In [115]:
tokenizer = AutoTokenizer.from_pretrained("bert_finetuned", return_tensor="pt")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained("bert_finetuned")
model = model.to(device)

In [116]:
len(sentences)

22193

In [117]:
device

'cuda:0'

In [118]:
pred_labels_bert = []

for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt").input_ids.to(device) 
    
    with torch.no_grad():
        logits = model(inputs).logits
    predicted_class_id = logits.argmax().item()
    pred_labels_bert.append(predicted_class_id)

In [119]:
print(len(pred_labels_bert))
print(len(sentences))

22193
22193


In [120]:
print(pred_labels_bert.count(1)/len(pred_labels_bert))

0.404947505970351


# KNN 

In [121]:
from gensim.models import Word2Vec
import numpy as np
import pickle

In [122]:
w2v_model = Word2Vec.load("word2vec_knn.model")
vocab=list(w2v_model.wv.key_to_index.keys())
loaded_model = pickle.load(open('knn_pickle', 'rb'))

In [123]:
def avg_w2vec(sentences, labels):
    """
    Average Word2Vec approach for creating a vector for a given sentence from the word embeddings of each words of the sentence.
    """

    transformed=[]
    lab = []
    for sentence, label in zip(sentences,labels):
        count=0
        vector=np.zeros(300)
        for word in sentence.split():
            if word in vocab:
                vector+=w2v_model.wv.get_vector(word)
                count+=1
        if count!=0:
            vector/=count
            transformed.append(vector)
            lab.append(label)
    return np.array(transformed), np.array(lab)

In [124]:
transformed_text, labels1= avg_w2vec(sentences, labels)

In [125]:
pred_labels_knn = loaded_model.predict(transformed_text)

In [126]:
print(len(pred_labels_knn))
print(len(sentences))

22145
22193


In [127]:
np.unique(pred_labels_knn, return_counts=True)[1][1]/len(pred_labels_knn)

0.18044705351095056